In [1]:
import requests
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
with open('vetoes_countries.json') as json_data:
    geometry_data = json.load(json_data)
for feat in geometry_data['features']:
    del feat["properties"]['brk_group']
for feat in geometry_data['features']:
    del feat["properties"]['formal_fr']
for feat in geometry_data['features']:
    del feat["properties"]['note_adm0']
for feat in geometry_data['features']:
    del feat["properties"]['note_brk']
for feat in geometry_data['features']:
    del feat["properties"]['name_alt']

In [4]:
df = pd.DataFrame.from_dict(json_normalize(geometry_data['features']), orient='columns')

In [5]:
#Ugh, uniform country names guys. I'll have to fix it so it matches my csv.
df['properties.name'] = df['properties.name'].str.replace('Lao PDR', 'Laos')
df['properties.name'] = df['properties.name'].str.replace('Korea', 'South Korea')

In [6]:
#Let's pull in our csv data
df2 = pd.read_csv('membership_veto.csv')

In [8]:
#Let's join 'em up
output = pd.merge(df2, df[['geometry.coordinates','geometry.type','properties.name']], left_on='impacted_country', right_on='properties.name', how='left')

In [9]:
output['properties.headline'] = output['properties.name']
output['properties.color'] = '#CC3333'
output['properties.article'] = output['properties.name'] + ' had their application to the UN vetoed by ' + output['veto_power']
output['properties.group_name'] = output['veto_power']
output['properties.group_id'] = 0

In [10]:
output['properties.group_name']

0     United States of America
1     United States of America
2     United States of America
3     United States of America
4     United States of America
5     United States of America
6                        China
7                       Russia
8                       Russia
9                       Russia
10                      Russia
11                      Russia
12                      Russia
13                      Russia
14                      Russia
15                      Russia
16                      Russia
17                      Russia
18                      Russia
19                      Russia
20                      Russia
21                      Russia
22                      Russia
23                      Russia
24                      Russia
25                      Russia
26                      Russia
27                      Russia
28                      Russia
29                      Russia
30                      Russia
31                      Russia
32      

In [11]:
output.loc[0:5, 'properties.group_id'] = 2
output.loc[6, 'properties.group_id'] = 3
output.loc[7:, 'properties.group_id'] = 1

In [19]:
output

,agenda,date,draft,draft_url,meeting_record,meeting_record_url,veto_power,veto_china,veto_france,veto_russia,...,agenda_cleaned,impacted_country,geometry.coordinates,geometry.type,properties.name,properties.headline,properties.color,properties.article,properties.group_name,properties.group_id
0,Admission of new Members Viet Nam,15 November 1976,S/12226,http://www.un.org/en/ga/search/view_doc.asp?sy...,S/PV.1972,http://www.un.org/en/ga/search/view_doc.asp?sy...,United States of America,NaN,NaN,NaN,...,Admission of new Members Viet Nam,Vietnam,"[[[[104.00473066500015, 10.450995184000135], [...",MultiPolygon,Vietnam,Vietnam,#CC3333,Vietnam had their application to the UN vetoed...,United States of America,2
1,Admission of new Members Angola,23 June 1976,S/12110,http://www.un.org/en/ga/search/view_doc.asp?sy...,S/PV.1932,http://www.un.org/en/ga/search/view_doc.asp?sy...,United States of America,NaN,NaN,NaN,...,Admission of new Members Angola,Angola,"[[[[13.982329549000069, -5.853285013999837], [...",MultiPolygon,Angola,Angola,#CC3333,Angola had their application to the UN vetoed ...,United States of America,2
2,Admission of new Members Democratic Republic o...,30 September 1975,S/11833,http://www.un.org/en/ga/search/view_doc.asp?sy...,S/PV.1846,http://www.un.org/en/ga/search/view_doc.asp?sy...,United States of America,NaN,NaN,NaN,...,Admission of new Members Democratic Republic o...,Vietnam,"[[[[104.00473066500015, 10.450995184000135], [...",MultiPolygon,Vietnam,Vietnam,#CC3333,Vietnam had their application to the UN vetoed...,United States of America,2
3,Admission of new Members Republic of South Vie...,30 September 1975,S/11832,http://www.un.org/en/ga/search/view_doc.asp?sy...,S/PV.1846,http://www.un.org/en/ga/search/view_doc.asp?sy...,United States of America,NaN,NaN,NaN,...,Admission of new Members Republic of South Vie...,Vietnam,"[[[[104.00473066500015, 10.450995184000135], [...",MultiPolygon,Vietnam,Vietnam,#CC3333,Vietnam had their application to the UN vetoed...,United States of America,2
4,Admission of new Members Democratic Republic o...,11 August 1975,S/11796,http://www.un.org/en/ga/search/view_doc.asp?sy...,S/PV.1836,http://www.un.org/en/ga/search/view_doc.asp?sy...,United States of America,NaN,NaN,NaN,...,Admission of new Members Democratic Republic o...,Vietnam,"[[[[104.00473066500015, 10.450995184000135], [...",MultiPolygon,Vietnam,Vietnam,#CC3333,Vietnam had their application to the UN vetoed...,United States of America,2
5,Admission of new Members Republic of South Vie...,11 August 1975,S/11795,http://www.un.org/en/ga/search/view_doc.asp?sy...,S/PV.1836,http://www.un.org/en/ga/search/view_doc.asp?sy...,United States of America,NaN,NaN,NaN,...,Admission of new Members Republic of South Vie...,Vietnam,"[[[[104.00473066500015, 10.450995184000135], [...",MultiPolygon,Vietnam,Vietnam,#CC3333,Vietnam had their application to the UN vetoed...,United States of America,2
6,Admission of new Members Bangladesh,25 August 1972,S/10771,http://www.un.org/en/ga/search/view_doc.asp?sy...,S/PV.1660,http://www.un.org/en/ga/search/view_doc.asp?sy...,China,China,NaN,NaN,...,Admission of new Members Bangladesh,Bangladesh,"[[[[91.9387313160001, 21.714829820000105], [91...",MultiPolygon,Bangladesh,Bangladesh,#CC3333,Bangladesh had their application to the UN vet...,China,3
7,Applications for Membership Kuwait,30 November 1961,S/5006,http://www.un.org/en/ga/search/view_doc.asp?sy...,S/PV.985,http://www.un.org/en/ga/search/view_doc.asp?sy...,Russia,NaN,NaN,NaN,...,Applications for Membership Kuwait,Kuwait,"[[[[48.18873131600011, 29.983547268000123], [4...",MultiPolygon,Kuwait,Kuwait,#CC3333,Kuwait had their application to the UN vetoed ...,Russia,1
8,Applications for membership Mauritania,3 December 1960,S/4567/Rev.1,http://www.un.org/en/ga/search/view_doc.asp?sy...,S/PV.911,http://www.un.org/en/ga/search/view_doc.asp?sy...,Russia,NaN,NaN,NaN,...,Applications for membership Mauritania,Mauritania,"[[[-6.148741413999886, 21.21953277600012],

In [75]:
def fixing_article(row):
    #print('----------------')
    if output.impacted_country.value_counts()[row.impacted_country] == 1:
        #print('1')
        draft_resolution = "Draft resolution: <a href=" + row.draft_url + ">" + row.draft + "</a><br>"
        meeting_notes = "Meeting notes: <a href=" + row.meeting_record_url+ ">" + row.meeting_record + "</a><br>"
        agenda_item = "Agenda item: " + row.agenda + "<br>"
        row['properties.article'] = "<b>" + row.impacted_country + " had their application for membership vetoed on " + row.date + " by " + row.veto_power + ".</b><br>" + draft_resolution + meeting_notes + agenda_item
    if output.impacted_country.value_counts()[row.impacted_country] > 1:
        #print("several"+row.impacted_country+row.draft)
        veto_number = output.impacted_country.value_counts()[row.impacted_country]
        #print(veto_number)
        article_string = ''
        for index, row_inner in output[output.impacted_country == row.impacted_country].iterrows():
            draft_resolution = "Draft resolution: <a href=" + row_inner.draft_url + ">" + row_inner.draft + "</a><br>"
            meeting_notes = "Meeting notes: <a href=" + row_inner.meeting_record_url+ ">" + row_inner.meeting_record + "</a><br>"
            agenda_item = "Agenda item: " + row_inner.agenda + "<br>"
            article_string += "<b>On " + row_inner.date + " by " + row_inner.veto_power + ".</b><br>" + draft_resolution + meeting_notes + agenda_item
        row['properties.article'] = row.impacted_country + " had their application for membership vetoed <b>" + str(veto_number) + "</b> times:<br>" + article_string
    return row

In [76]:
df_final = output.apply(fixing_article, axis=1)

In [77]:
ok_json = json.loads(df_final.to_json(orient='records'))

In [78]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [79]:
geo_format = process_to_geojson(ok_json)

In [80]:
with open('geo-data.js', 'w') as outfile:
    json.dump(geo_format, outfile)

In [81]:
df_final.to_csv('membership_veto_final.csv', index=False)